# Example Notebook for transport measurements 

In [ ]:
import numpy
import time
import qkit
from qkit.measure.transport import transport
import qkit.measure.samples_class as sc

## starting qkit

In [ ]:
#qkit.cfg['datafolder_structure']=2
qkit.cfg['run_id']='run_number'
qkit.cfg['user']='User'
qkit.start()

### create measurement sample

In [ ]:
sample = sc.Sample()
sample.name = 'A0000#0'
sample.comment = 'This sample looks promising.'
sample.ports = 'I0to0_V0to0'
print(sample.get_all())

# Initialize transport and IV device

In [ ]:
#Select and initialize Multi Channel Source Unit:
#either
IVD = qkit.instruments.create('IVD', 'Keithley', address='TCPIP0::00.00.000.00::INSTR', reset=True) #Keithley_2636A
#or
#IVD = qkit.instruments.create('IVD2', 'Yokogawa', address='TCPIP0::00.00.000.00::INSTR', reset=True) #Yokogawa_GS820

#Microwave source:
mw_src = qkit.instruments.create('mw_src','Anritsu_MG37022', address = 'TCPIP0::00.00.000.00::inst0::INSTR')

#Temperature control
tip = qkit.instruments.create('tip', 'TIP_client', address='IP address', port='PORT')

In [ ]:
#Sample and port numbers:
exp = sample.name+'_'+sample.ports # Sample_ports

In [ ]:
#Reset and get settings 
IVD.reset()
IVD.clear_error()
IVD.set_defaults()
IVD.get_all(channel=1)

# 1D Scan

In [ ]:
#####################################################################################################
########################################## Current bias 1D ##########################################
#####################################################################################################

comment="""%s: I-V curve""" % exp #set before - sample and ports for comment and dir

nplc = 0 #Number of power line cycles
avg = 0 #Average
delay = 15e-6 #measurement delay

tr.comment = comment
dirname = exp

dAdV = 0 #Voltage biased current source -> conversion factor (U -> I)
amp = 0 #Preamplifier factor (e.g. 1e3)
IVD.set_dAdV(dAdV)
IVD.set_amp(amp)
IVD.set_pseudo_bias_mode(0)

#for voltage bias:
#dVdA = .... amplification factor for voltage bias (e.g. 1e9) instead of dAdV 
#Vdiv = .... external voltage divider factor (e.g. 1e3) instead of amp
#IVD.set_dVdA(dVdA) instead of IVD.set_dAdV(dAdV)
#IVD.set_Vdiv(Vdiv) insteal of IVD.set_amp(amp)
#IVD.set_pseudo_bias_mode(1) instead of IVD.set_pseudo_bias_mode(0) 

IVD.set_sweep_mode(0) # 0 (VV-mode) | 1 (IV-mode) | 2 (VI-mode)
IVD.set_pseudo_bias_mode(0) # 0 (current bias) | 1 (voltage bias)
IVD.set_sense_nplc(nplc, 1)
IVD.set_sense_nplc(nplc, 2)


tr.set_dVdI(True)
tr.sweep.reset_sweeps()
tr.add_sweep_4quadrants(start=0, stop=0, step=0, offset=0) # 4 quadrants sweep: start at zero -> stop -> zero -> -stop -> zero
#tr.sweep.add_sweep(start=0, stop=0, step=0) # single sweep


In [ ]:
tr.measure_1D() # start 1D scan

# 2D scan

In [ ]:
####################################################################################
############################## Current bias 2D #####################################
####################################################################################


comment="""%s: I-V curve""" % exp #set before - sample and ports for comment and dir

nplc = 0 #Number of power line cycles
avg = 0 #Average
delay = 15e-6 #measurement delay

tr.comment = comment
dirname = exp

dAdV = 0 #Voltage biased current source -> conversion factor (U -> I)
amp = 0 #Preamplifier factor (e.g. 1e3)
IVD.set_dAdV(dAdV)
IVD.set_amp(amp)
IVD.set_pseudo_bias_mode(0)

#for voltage bias:
#dVdA = .... amplification factor for voltage bias (e.g. 1e9) instead of dAdV 
#Vdiv = .... external voltage divider factor (e.g. 1e3) instead of amp
#IVD.set_dVdA(dVdA) instead of IVD.set_dAdV(dAdV)
#IVD.set_Vdiv(Vdiv) insteal of IVD.set_amp(amp)
#IVD.set_pseudo_bias_mode(1) instead of IVD.set_pseudo_bias_mode(0) 

IVD.set_sweep_mode(0) # 0 (VV-mode) | 1 (IV-mode) | 2 (VI-mode)
IVD.set_pseudo_bias_mode(0) # 0 (current bias) | 1 (voltage bias)
IVD.set_sense_nplc(nplc, 1)
IVD.set_sense_nplc(nplc, 2)

tr.set_dVdI(True)
tr.sweep.reset_sweeps()
tr.add_sweep_4quadrants(start=0, stop=0, step=0, offset=0) # 4 quadrants sweep: start at zero -> stop -> zero -> -stop -> zero
#tr.sweep.add_sweep(start=0, stop=0, step=0) # single sweep


#example for 2D scan - power scan:
mw_src.set_status(True)
'''mw power scan'''
freq = 0. # Set frequency 
mw_src.set_frequency(freq)

def x_function(i):
    mw_src.enable_high_power = False
    mw_src.set_power(i)
    time.sleep(0.01)
    return

tr.set_x_parameters(x_vec=numpy.arange(0,0,0), # (start,stop,step) 
                    x_coordname='power', 
                    x_set_obj=x_function, 
                    x_unit ='dBm')


#log function, e.g. temperature tracking
def get_T():
    return tip.get_T()

tr.set_log_function([get_T], ['temp'], ['K'], ['f'])

In [ ]:
tr.measure_2D()

# 3D scan

In [ ]:
#####################################################################################################
########################################## Current bias 3D #############################################
#####################################################################################################

comment="""%s: I-V curve""" % exp #set before - sample and ports for comment and dir

nplc = 0 #Number of power line cycles
avg = 0 #Average
delay = 15e-6 #measurement delay

tr.comment = comment
dirname = exp

dAdV = 0 #Voltage biased current source -> conversion factor (U -> I)
amp = 0 #Preamplifier factor (e.g. 1e3)
IVD.set_dAdV(dAdV)
IVD.set_amp(amp)
IVD.set_pseudo_bias_mode(0)

#for voltage bias:
#dVdA = .... amplification factor for voltage bias (e.g. 1e9) instead of dAdV 
#Vdiv = .... external voltage divider factor (e.g. 1e3) instead of amp
#IVD.set_dVdA(dVdA) instead of IVD.set_dAdV(dAdV)
#IVD.set_Vdiv(Vdiv) insteal of IVD.set_amp(amp)
#IVD.set_pseudo_bias_mode(1) instead of IVD.set_pseudo_bias_mode(0) 

IVD.set_sweep_mode(0) # 0 (VV-mode) | 1 (IV-mode) | 2 (VI-mode)
IVD.set_pseudo_bias_mode(0) # 0 (current bias) | 1 (voltage bias)
IVD.set_sense_nplc(nplc, 1)
IVD.set_sense_nplc(nplc, 2)

tr.set_dVdI(True)
tr.sweep.reset_sweeps()
tr.add_sweep_4quadrants(start=0, stop=0, step=0, offset=0) # 4 quadrants sweep: start at zero -> stop -> zero -> -stop -> zero
#tr.sweep.add_sweep(start=0, stop=0, step=0) # single sweep

#example for 3D mw scan - vary power and frequency:
mw_src.set_status(True)
'''mw power scan'''
freq = 0. # Set frequency 
mw_src.set_frequency(freq)

# power variation:
def x_function(i):
    mw_src.enable_high_power = False
    mw_src.set_power(i)
    time.sleep(0.01)
    return

tr.set_x_parameters(x_vec=numpy.arange(0,0,0), # (start,stop,step) 
                    x_coordname='power', 
                    x_set_obj=x_function, 
                    x_unit ='dBm')

# frequency variation
def y_function(i):
    mw_src.set_status(True)
    time.sleep(0.1)
    mw_src.set_frequency(i)
    return 

tr.set_y_parameters(y_vec=numpy.arange(0.0e9, 20.0e9, 1.0e9), #  (start, stop, step)        
                    y_coordname='mw_frequency', 
                    y_set_obj=y_function, 
                    y_unit ='Hz')


#log function, e.g. temperature tracking
def get_T():
    return tip.get_T()

tr.set_log_function([get_T], ['temp'], ['K'], ['f'])

In [ ]:
tr.measure_3D()

# stop qkit

In [ ]:
IVD.close()

In [ ]:
exit()